In [ ]:
import folium
from folium import plugins

from srai.datasets import AirbnbMulticity

%load_ext dotenv
%dotenv

In [ ]:
airbnb_multicity = AirbnbMulticity()

In [ ]:
airbnb_multicity_gdf = airbnb_multicity.load()

In [ ]:
airbnb_multicity_gdf.head()

In [ ]:
# visualitzation
map = folium.Map(location=[15, 30], tiles="Cartodb dark_matter", zoom_start=2)

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in airbnb_multicity_gdf.geometry]

plugins.HeatMap(heat_data).add_to(map)

In [ ]:
map